In [1]:
# Class label csv path
labels_csv_path = 'datasets/kinetics_400_labels.csv'

In [2]:
from msclap import CLAP
import torch.nn.functional as F
import numpy as np
import torch
import os
from moviepy.editor import VideoFileClip
import sounddevice as sd
import numpy as np
from torch.nn import functional as F
from scipy.io.wavfile import write
import time
import threading
import queue
from IPython.display import display, clear_output

In [3]:
import csv

label2id = {}
id2label = {}
with open(labels_csv_path, mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)

    for row in csv_reader:
        class_id, class_name = row
        label2id[class_name] = class_id
        id2label[class_id] = class_name

class_labels = list(label2id.keys())
class_labels.append("glass breaking")

# print(f"{len(class_labels)} Unique classes: {class_labels}.")

In [4]:
# Set the duration and sample rate
window_size = 10  # seconds
sample_rate = 44100  # Hz

# Create a queue to hold the audio data
audio_queue = queue.Queue()

# Function to record audio continuously and add it to the queue
def record_audio():
    while True:
        recording = sd.rec(int(window_size * sample_rate), samplerate=sample_rate, channels=1)
        sd.wait()  # Wait until recording is finished
        audio_queue.put(recording)
        print(f"Current queue size: {audio_queue.qsize()}")

# Start the recording thread
recording_thread = threading.Thread(target=record_audio)
recording_thread.start()

print("Loading model...")
# Load model (Choose between versions '2022' or '2023')
# The model weight will be downloaded automatically if `model_fp` is not specified
clap_model = CLAP(version='2023', use_cuda=False)

print("Extracting text embeddings...")
# Extract text embeddings
text_embeddings = clap_model.get_text_embeddings([f"This is a sound of {c}" for c in class_labels])

print("Listening...")

while True:    
    # Get the latest audio data from the queue
    recording = audio_queue.get()

    # Save the recording to a temporary file
    temp_filename = "temp_recording.wav"
    write(temp_filename, sample_rate, recording)

    # Extract audio embeddings
    audio_embeddings = clap_model.get_audio_embeddings([temp_filename])
    # Compute similarity between audio and text embeddings
    similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)

    similarity = F.softmax(similarities, dim=1)
    values, indices = similarity[0].topk(5)
    
    # Check if "glass breaking" is in the top five predictions
    things_to_detect = ["glass breaking", "using computer"]
    detected = any(class_labels[index] in things_to_detect for index in indices)
    
    print("Top predictions:\n" + "\n".join(f"{class_labels[index]:>16s}: {100 * value.item():.2f}%" for value, index in zip(values, indices)) + "\n")
    
    if detected:
        filename = f"recording_{int(time.time())}.wav"
        os.rename(temp_filename, filename)
        print(f"Alert! Glass breaking detected in {filename}")
    else:
#         print("No glass breaking detected in this recording.")
        os.remove(temp_filename) 
    

Loading model...
Extracting text embeddings...
Current queue size: 1
Current queue size: 2
Listening...
Top predictions:
answering questions: 67.25%
   tying bow tie: 2.58%
   playing poker: 2.46%
  massaging feet: 1.92%
      testifying: 1.91%

Top predictions:
answering questions: 13.09%
   tying bow tie: 12.26%
tying knot (not on a tie): 10.58%
filling eyebrows: 7.84%
  massaging feet: 7.68%

Current queue size: 1
Top predictions:
    tasting beer: 9.43%
     eating cake: 7.90%
tying knot (not on a tie): 7.52%
        drinking: 6.89%
   tying bow tie: 4.86%

Current queue size: 1
Top predictions:
      testifying: 15.66%
answering questions: 9.43%
assembling computer: 7.82%
   tying bow tie: 7.17%
tying knot (not on a tie): 5.74%

Current queue size: 1
Top predictions:
answering questions: 34.11%
         hugging: 27.14%
   setting table: 3.66%
   tying bow tie: 3.12%
    tasting beer: 2.38%

Current queue size: 1
Top predictions:
answering questions: 57.89%
   playing poker: 18.41%

KeyboardInterrupt: 